In [1]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-81c822c5-d1f8-0947-cb9a-1bab68587b9c)


In [2]:
%%time

# # both
# !pip install -qqq transformers==4.31.0 --progress-bar off
# !pip install -qqq langchain==0.0.266 --progress-bar off
# !pip install -qqq xformers==0.0.20 --progress-bar off
# !pip install -qqq sentence_transformers==2.2.2 --progress-bar off
# !pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off

# th
# !pip install -Uqqq pip --progress-bar off
# !pip install -qqq torch==2.0.1 --progress-bar off
!pip install chromadb --progress-bar off
# !pip install -qqq pypdf==3.15.0 --progress-bar off
# !pip install -qqq pdf2image==1.16.3 --progress-bar off
!pip install replicate


# my
! pip install -qq -U faiss-cpu
# ! pip install -qq -U accelerate bitsandbytes einops tiktoken

# both
!pip install transformers --progress-bar off
!pip install langchain --progress-bar off
# !pip install xformers --progress-bar off
!pip install sentence_transformers --progress-bar off
!pip install InstructorEmbedding --progress-bar off

CPU times: user 269 ms, sys: 31.2 ms, total: 300 ms
Wall time: 41 s


In [3]:
# !pip uninstall -y torch

In [4]:
import torch
import torchvision

In [5]:
torch.cuda.is_available()

True

In [6]:
# pip install auto-gptq

In [7]:
# !wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.1/auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl

In [8]:
# !pip install -qqq auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off

In [9]:
# !sudo apt-get install poppler-utils

In [10]:
import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time

import langchain

# loaders
# from langchain.document_loaders import PyPDFLoader
# from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader


# splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

# prompts
from langchain import PromptTemplate, LLMChain

# vector stores
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma

# models
from langchain.llms import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import Replicate

# retrievers
from langchain.chains import RetrievalQA

import torch
import transformers
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# from transformers import AutoTokenizer, TextStreamer, pipeline


print(langchain.__version__)

0.0.325


In [11]:
# from langchain.vectorstores import Chroma
# from auto_gptq import AutoGPTQForCausalLM
# import torch
# from langchain import HuggingFacePipeline, PromptTemplate
# from langchain.chains import RetrievalQA
# from langchain.document_loaders import PyPDFDirectoryLoader
# from langchain.embeddings import HuggingFaceInstructEmbeddings
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from pdf2image import convert_from_path
# from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda:0


In [12]:
# max_len = 8192

In [13]:
# class CFG:
#     # LLMs
#     model_name = 'llama2-13b' # wizardlm, bloom, falcon, llama2-7b, llama2-13b
#     temperature = 0,
#     top_p = 0.95,
#     repetition_penalty = 1.15

#     # splitting
#     split_chunk_size = 800
#     split_overlap = 0

#     # embeddings
#     embeddings_model_repo = 'hkunlp/instructor-base'

#     # similar passages
#     k = 3

In [14]:
# model_repo = 'daryl149/llama-2-13b-chat-hf'

# tokenizer = AutoTokenizer.from_pretrained(model_repo, use_fast=True)

# model = AutoModelForCausalLM.from_pretrained(
#     model_repo,
#     load_in_4bit=True,
#     device_map='auto',
#     torch_dtype=torch.float16,
#     low_cpu_mem_usage=True,
#     trust_remote_code=True
# )

# max_len = 8192

In [15]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda:0


In [16]:
# model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
# model_basename = "model"

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

# model = AutoGPTQForCausalLM.from_quantized(
#     model_name_or_path,
#     revision="gptq-4bit-128g-actorder_True",
#     model_basename=model_basename,
#     use_safetensors=True,
#     trust_remote_code=True,
#     inject_fused_attention=False,
#     device=DEVICE,
#     quantize_config=None,
# )

In [17]:
# streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [18]:
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=1024,
#     temperature=0,
#     top_p=0.95,
#     repetition_penalty=1.15,
#     streamer=streamer,
# )

# llm = HuggingFacePipeline(pipeline = pipe, model_kwargs={"temperature": 0})

In [37]:
REPLICATE_API_TOKEN = "r8_4o6DI4Kl9VfQdrVv6OlaqvAyMhFdamr2jUDVe"
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

llm = Replicate(
    model = "replicate/llama-2-70b-chat:2796ee9483c3fd7aa2e171d38f4ca12251a30609463dcfd4cd76703f22e96cdf",
    input = {"temperature": 0.75, "max_length": 1024, "top_p": 0.95, "repetition_penalty": 1.15},
)

In [35]:
%%time
### testing model, not using the book yet
### answer is not necessarily related to the book
query = "Give me 5 examples of cool potions and explain what they do"
llm(query)

CPU times: user 486 ms, sys: 51.3 ms, total: 537 ms
Wall time: 1min 9s


' Sure, I\'d be happy to help! Here are five examples of cool potions and their effects:\n\n1. "Elixir of Life": This potion grants the player an extra life. When consumed, it will restore the player\'s health to full and also grant them temporary invincibility for a short period of time.\n2. "Potion of Speed": This potion increases the player\'s movement speed for a short period of time. It\'s useful for avoiding danger or chasing after enemies quickly.\n3. "Healing Potion": This pot'

In [21]:
prompt_template = """
Don't try to make up an answer, if you don't know just say that you don't know.
Answer in the same language the question was asked.
Use only the following pieces of context to answer the question at the end.

{context}

Question: {question}
Answer:"""


PROMPT = PromptTemplate(
    template = prompt_template,
    input_variables = ["context", "question"]
)

In [22]:
def loadForEmbeddings(book_txt):
    # load document
    loader = TextLoader(book_txt, encoding="utf-8")
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 800,
        chunk_overlap = 0
    )

    texts = text_splitter.split_documents(documents)
    return texts


In [23]:
def wrap_text_preserve_newlines(text, width=200): # 110
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_llm_response(llm_response):
    ans = wrap_text_preserve_newlines(llm_response['result'])

    sources_used = llm_response['source_documents'][0].metadata['source']

    ans = ans + '\n\nSources: \n' + sources_used
    return ans

In [24]:
def llm_ans(query):
    start = time.time()
    qa_chain = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
        retriever = retriever,
        chain_type_kwargs = {"prompt": PROMPT},
        return_source_documents = True,
        verbose = False
    )
    llm_response = qa_chain(query)
    ans = process_llm_response(llm_response)
    end = time.time()

    time_elapsed = int(round(end - start, 0))
    time_elapsed_str = f'\n\nTime elapsed: {time_elapsed} s'
    return ans + time_elapsed_str

In [25]:
import requests
from bs4 import BeautifulSoup
import difflib

# Function to search for a book by name and return the best match URL
def search_book_by_name(book_name):
    base_url = "https://www.gutenberg.org/"
    search_url = base_url + "ebooks/search/?query=" + book_name.replace(" ", "+") + "&submit_search=Go%21"

    response = requests.get(search_url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the best match link based on similarity ratio
    best_match_ratio = 0
    best_match_url = ""

    for link in soup.find_all("li", class_="booklink"):
        link_title = link.find("span", class_="title").get_text()
        similarity_ratio = difflib.SequenceMatcher(None, book_name.lower(), link_title.lower()).ratio()
        if similarity_ratio > best_match_ratio:
            best_match_ratio = similarity_ratio
            best_match_url = base_url + link.find("a").get("href")

    return best_match_url

# Function to get the "Plain Text UTF-8" download link from the book page
def get_plain_text_link(book_url):
    response = requests.get(book_url)
    soup = BeautifulSoup(response.content, "html.parser")

    plain_text_link = ""

    for row in soup.find_all("tr"):
        format_cell = row.find("td", class_="unpadded icon_save")
        if format_cell and "Plain Text UTF-8" in format_cell.get_text():
            plain_text_link = format_cell.find("a").get("href")
            break

    return plain_text_link


# Function to get the content of the "Plain Text UTF-8" link
def get_plain_text_content(plain_text_link):
    response = requests.get(plain_text_link)
    content = response.text
    return content


# Main function
def load_book(book_name):
    best_match_url = search_book_by_name(book_name)

    if best_match_url:
        plain_text_link = get_plain_text_link(best_match_url)
        if plain_text_link:
            full_plain_text_link = "https://www.gutenberg.org" + plain_text_link
            plain_text_content = get_plain_text_content(full_plain_text_link)
#             print("Plain Text UTF-8 content:", plain_text_content)

            book_text = plain_text_content

            file = book_name + ".txt"

            # Remove the BOM character if it exists
            book_text = book_text.lstrip('\ufeff')

            # Choose an appropriate encoding, such as 'utf-8'
            with open(file, "w", encoding="utf-8") as file:
                file.write(book_text)

            return book_text
        else:
            print("No Plain Text UTF-8 link found.")
            return "web site error"
    else:
        print("No matching book found.")
        return "web site error"

# def clean_book_content(book_text):
#     cleaned_book_text = book_text.replace("\n", " ")
#     cleaned_book_text = cleaned_book_text.replace("\r", " ")
#     cleaned_book_text = cleaned_book_text.replace("\ufeff", "")
#     return cleaned_book_text

In [ ]:
# !rm -rf "db"

In [ ]:
# !pip install chromadb --progress-bar off

In [26]:
# from langchain.vectorstores import Chroma
# from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS

In [27]:
book_name = "The prince"
book_text = load_book(book_name)
file = book_name + ".txt"
texts = loadForEmbeddings(file)

### download embeddings model
instructor_embeddings = HuggingFaceInstructEmbeddings(
        model_name = "hkunlp/instructor-base",
        model_kwargs = {"device": "cuda"}
    )
# embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# ### create embeddings and DB
vectordb = FAISS.from_documents(
    documents = texts,
    embedding = instructor_embeddings
)

### persist vector database
# vectordb.save_local("faiss_index_hp")

retriever = vectordb.as_retriever(search_kwargs = {"k": 3, "search_type" : "similarity"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
%%time
# vectordb = Chroma.from_documents(texts, embeddings, persist_directory="db")

In [ ]:
# # persistent_client = chromadb.PersistentClient(Configuration.Persist_directory)
# print("Creating book embeddings...")
# # book_embeddings = Chroma(
# #     collection_name=f"BabblerEmbedding-{book_id}",
# #     embedding_function=embeddings,
# #     client=persistent_client,
# #     persist_directory=Configuration.Persist_directory,
# # )
# book_embeddings = Chroma(
#     embedding_function=embeddings,
# )

In [ ]:
# book_embeddings.add_documents(documents=texts)

In [ ]:
# retriever = vectordb.as_retriever(search_kwargs = {"k": 3, "search_type" : "similarity"})

In [39]:
%%time
llm_ans("What are the characters in this story")

CPU times: user 137 ms, sys: 8.7 ms, total: 146 ms
Wall time: 14.2 s


' The characters mentioned in the passage are:\n\n* Zanobi\n* Luigi\n* The people who have performed great deeds in the world and excelled all others in their day\n* Those who have been aggrieved by Fortune in some outrageous way\n* The persons who have been exposed to the mercy of wild beasts\n* The individuals who have given themselves out to be sons of Jove or of some other deity.\n\nSources: \nThe prince.txt\n\nTime elapsed: 14 s'

In [41]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 52.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.6 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl size=5579 sha256=a5c63c24eea10535efea0b2482afd149f2b5392cdcd0791a359c261b6e89cabb
  Stored in directory: /root/.cache/pip/wheels/01/a6/d1/1c0828c304a4283b2c1639a09ad86f83d7c487ef34c6b4a1bf
Successfully built ffmpy
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

In [42]:
import gradio as gr

title = "GutenbergChat Hub"
vectordb = None
retriever = None

# Submit book
def submit_book(book_name):
    global vectordb, retriever, instructor_embeddings
    if not book_name:
        return "Please enter the name of the book."

    book_text = load_book(book_name)
    file = book_name + ".txt"
    texts = loadForEmbeddings(file)

    # ### download embeddings model
    # instructor_embeddings = HuggingFaceInstructEmbeddings(
    #     model_name = 'hkunlp/instructor-base',
    #     model_kwargs = {"device": "cuda"}
    # )

    ### create embeddings and DB
    vectordb = FAISS.from_documents(
        documents = texts,
        embedding = instructor_embeddings
    )

    # ### persist vector database
    # vectordb.save_local("faiss_index_hp")

    retriever = vectordb.as_retriever(search_kwargs = {"k": 3, "search_type" : "similarity"})


    return "done"


# Q/A
def get_response(prompt):
#     if (not book_name and not prompt):
#         return "Please enter the name of the book and the prompt."
#     if not book_name:
#         return "Please enter the name of the book."
#     if not prompt:
#         return "Please enter the prompt."
    query = prompt
#     llm_response = qa_chain(query)

    return llm_ans(query)

# Interface 1
submitBook = gr.Interface(fn=submit_book, inputs="text", outputs="text", title="Submit your book here first")
# Interface 2
chatBot = gr.Interface(
        fn=get_response,
        inputs="text",
        outputs="text",
        title=title + " - Q/A-Bot",
        description="Enter the name of the book in previous tab, then ask questions here",
        examples=["What are the characters in the book?"]
    )

demo = gr.TabbedInterface([submitBook, chatBot], ["SubmitBook", "Q/A-Bot"])
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d42218ab8867e52f4f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
